In [72]:
import MDAnalysis as mda
import numpy as np

In [104]:

u = mda.Universe('file.gro')
c = np.unique(u.atoms.positions[:,1]) # finding the unique y-coordinate so that we can loop through each row of atoms
u.atoms.masses = 36    # Since it's a TC5 bead, we use a mass of 36 for each

In [105]:
for j in range(len(c)): # Looping through each row, defined by the y-coordinate
    if (j !=0) and (j !=len(c)-1):
        if j % 2 !=0:
            group = u.atoms[u.atoms.positions[:,1] == c[j]]
            gr = np.arange(1, len(group), 3)
         
            for k in gr:
                u.atoms[group[k].index].mass = 0
            
        else:
            group = u.atoms[u.atoms.positions[:,1] == c[j]]
            gr = np.arange(2, len(group), 3)
            for k in gr:
                u.atoms[group[k].index].mass = 0
                

In [106]:
v = u.atoms[u.atoms.masses !=0]

In [107]:
def hexagon(universe):
    
    ''' Identifying the vertics of hexagon around a virtual site '''
    
    b = u.atoms[u.atoms.masses == 0].indices
    hexagon_indices = []
    for i in b:
        empty = []
        for j in u.atoms.indices:
            if (2.55 <= calc_bonds(u.atoms[j].position, u.atoms[i].position) <= 2.57):
                empty.append(j)
        hexagon_indices.append(empty)
    return hexagon_indices

In [108]:
from MDAnalysis.lib.distances import calc_angles, calc_bonds, calc_dihedrals

In [109]:
hexagons = hexagon(u)

In [110]:
def get_angles(hexagons):
    angles = []
    for hex in hexagons:
        angle1 = [hex[2], hex[4], hex[0], hex[5]]
        angle2 = [hex[4], hex[0], hex[5], hex[1]]
        angle3 = [hex[0], hex[5], hex[1], hex[3]]
        angles.append(angle1)
        angles.append(angle2)
        angles.append(angle3)
    return angles

In [111]:
angles = get_angles(hexagons)

In [112]:
angles = np.array(angles)
angles

array([[ 6, 15,  0, 16],
       [15,  0, 16,  1],
       [ 0, 16,  1,  8],
       [ 9, 18,  2, 19],
       [18,  2, 19,  3],
       [ 2, 19,  3, 11],
       [12, 21,  4, 22],
       [21,  4, 22,  5],
       [ 4, 22,  5, 14],
       [16, 25,  8, 26],
       [25,  8, 26,  9],
       [ 8, 26,  9, 18],
       [19, 28, 11, 29],
       [28, 11, 29, 12],
       [11, 29, 12, 21],
       [23, 32, 15, 33],
       [32, 15, 33, 16],
       [15, 33, 16, 25],
       [26, 34, 18, 35],
       [34, 18, 35, 19],
       [18, 35, 19, 28],
       [29, 36, 21, 37],
       [36, 21, 37, 22],
       [21, 37, 22, 31]])

In [113]:
def get_between_hexagons_angles(hexagons, beads_per_row = 9):
    def get_common_side(hexa1, hexa2):
        common = list(set(hexa1).intersection(set(hexa2)))
        common.sort()
        return common
    
    angles = []
    gap_for_above = int(beads_per_row / 2)
    for i in range(len(hexagons)):
        hexa = hexagons[i]
        try:
            hexa_after = hexagons[i+2]
        except IndexError:
            hexa_after = None
        try: 
            hexa_above = hexagons[i+gap_for_above]
        except IndexError:
            hexa_above = None
        
        try: 
            hexa_before = hexagons[i - 1]
        except IndexError:
            hexa_before = None
        
        if hexa_after:
            side_after = get_common_side(hexa, hexa_after)
            if side_after:
                idx1 = hexa.index(side_after[0])
                idx2 = hexa_after.index(side_after[1])
                angle = [hexa[idx1 - 2], side_after[0], side_after[1], hexa_after[idx2 + 2]]
                angles.append(angle)
            
        
        if hexa_above: 
            side_above = get_common_side(hexa, hexa_above)
            if side_above: 
                idx1 = hexa.index(side_above[0])
                idx2 = hexa_above.index(side_above[1])
                angle = [hexa[idx1 - 4], side_above[0], side_above[1], hexa_above[idx2 + 4]]
                angles.append(angle)  
                
        if hexa_before:
            side_before = get_common_side(hexa, hexa_before)
            if side_before:
                idx1 = hexa_before.index(side_before[0])
                idx2 = hexa.index(side_before[1])
                angle = [hexa_before[idx1 - 2], side_before[0], side_before[1], hexa[idx2 + 2]]
                angles.append(angle)
    return angles

In [114]:
other_angles = get_between_hexagons_angles(hexagons)

In [115]:
other_angles

[[2, 9, 18, 26], [4, 12, 21, 29], [8, 16, 25, 33], [11, 19, 28, 35]]

[[6, 15, 0, 16],
 [15, 0, 16, 1],
 [0, 16, 1, 8],
 [9, 18, 2, 19],
 [18, 2, 19, 3],
 [2, 19, 3, 11],
 [12, 21, 4, 22],
 [21, 4, 22, 5],
 [4, 22, 5, 14],
 [16, 25, 8, 26],
 [25, 8, 26, 9],
 [8, 26, 9, 18],
 [19, 28, 11, 29],
 [28, 11, 29, 12],
 [11, 29, 12, 21],
 [23, 32, 15, 33],
 [32, 15, 33, 16],
 [15, 33, 16, 25],
 [26, 35, 18, 36],
 [35, 18, 36, 19],
 [18, 36, 19, 28],
 [29, 38, 21, 39],
 [38, 21, 39, 22],
 [21, 39, 22, 31],
 [33, 42, 25, 43],
 [42, 25, 43, 26],
 [25, 43, 26, 35],
 [36, 45, 28, 46],
 [45, 28, 46, 29],
 [28, 46, 29, 38],
 [40, 49, 32, 50],
 [49, 32, 50, 33],
 [32, 50, 33, 42],
 [43, 52, 35, 53],
 [52, 35, 53, 36],
 [35, 53, 36, 45],
 [46, 55, 38, 56],
 [55, 38, 56, 39],
 [38, 56, 39, 48],
 [50, 59, 42, 60],
 [59, 42, 60, 43],
 [42, 60, 43, 52],
 [53, 62, 45, 63],
 [62, 45, 63, 46],
 [45, 63, 46, 55],
 [57, 66, 49, 67],
 [66, 49, 67, 50],
 [49, 67, 50, 59],
 [60, 68, 52, 69],
 [68, 52, 69, 53],
 [52, 69, 53, 62],
 [63, 70, 55, 71],
 [70, 55, 71, 56],
 [55, 71, 56, 65

In [53]:
other_angles

[[2, 9, 18, 26],
 [4, 12, 21, 29],
 [8, 16, 25, 33],
 [11, 19, 28, 36],
 [18, 26, 35, 43],
 [21, 29, 38, 46],
 [25, 33, 42, 50],
 [28, 36, 45, 53],
 [35, 43, 52, 60],
 [38, 46, 55, 63],
 [42, 50, 59, 67],
 [45, 53, 62, 69]]

In [56]:
agg = np.array(other_angles)

In [58]:
agg = agg+1

In [59]:
agg

array([[ 3, 10, 19, 27],
       [ 5, 13, 22, 30],
       [ 9, 17, 26, 34],
       [12, 20, 29, 37],
       [19, 27, 36, 44],
       [22, 30, 39, 47],
       [26, 34, 43, 51],
       [29, 37, 46, 54],
       [36, 44, 53, 61],
       [39, 47, 56, 64],
       [43, 51, 60, 68],
       [46, 54, 63, 70]])

In [64]:
angles

array([[ 6, 15,  0, 16],
       [15,  0, 16,  1],
       [ 0, 16,  1,  8],
       [ 9, 18,  2, 19],
       [18,  2, 19,  3],
       [ 2, 19,  3, 11],
       [12, 21,  4, 22],
       [21,  4, 22,  5],
       [ 4, 22,  5, 14],
       [16, 25,  8, 26],
       [25,  8, 26,  9],
       [ 8, 26,  9, 18],
       [19, 28, 11, 29],
       [28, 11, 29, 12],
       [11, 29, 12, 21],
       [23, 32, 15, 33],
       [32, 15, 33, 16],
       [15, 33, 16, 25],
       [26, 35, 18, 36],
       [35, 18, 36, 19],
       [18, 36, 19, 28],
       [29, 38, 21, 39],
       [38, 21, 39, 22],
       [21, 39, 22, 31],
       [33, 42, 25, 43],
       [42, 25, 43, 26],
       [25, 43, 26, 35],
       [36, 45, 28, 46],
       [45, 28, 46, 29],
       [28, 46, 29, 38],
       [40, 49, 32, 50],
       [49, 32, 50, 33],
       [32, 50, 33, 42],
       [43, 52, 35, 53],
       [52, 35, 53, 36],
       [35, 53, 36, 45],
       [46, 55, 38, 56],
       [55, 38, 56, 39],
       [38, 56, 39, 48],
       [50, 59, 42, 60],


In [66]:
type(angles)

numpy.ndarray

In [68]:
other_angles = np.array(other_angles) + 1

In [69]:
other_angles

array([[ 3, 10, 19, 27],
       [ 5, 13, 22, 30],
       [ 9, 17, 26, 34],
       [12, 20, 29, 37],
       [19, 27, 36, 44],
       [22, 30, 39, 47],
       [26, 34, 43, 51],
       [29, 37, 46, 54],
       [36, 44, 53, 61],
       [39, 47, 56, 64],
       [43, 51, 60, 68],
       [46, 54, 63, 70]])

In [70]:
np.vstack((angles, other_angles))

array([[ 6, 15,  0, 16],
       [15,  0, 16,  1],
       [ 0, 16,  1,  8],
       [ 9, 18,  2, 19],
       [18,  2, 19,  3],
       [ 2, 19,  3, 11],
       [12, 21,  4, 22],
       [21,  4, 22,  5],
       [ 4, 22,  5, 14],
       [16, 25,  8, 26],
       [25,  8, 26,  9],
       [ 8, 26,  9, 18],
       [19, 28, 11, 29],
       [28, 11, 29, 12],
       [11, 29, 12, 21],
       [23, 32, 15, 33],
       [32, 15, 33, 16],
       [15, 33, 16, 25],
       [26, 35, 18, 36],
       [35, 18, 36, 19],
       [18, 36, 19, 28],
       [29, 38, 21, 39],
       [38, 21, 39, 22],
       [21, 39, 22, 31],
       [33, 42, 25, 43],
       [42, 25, 43, 26],
       [25, 43, 26, 35],
       [36, 45, 28, 46],
       [45, 28, 46, 29],
       [28, 46, 29, 38],
       [40, 49, 32, 50],
       [49, 32, 50, 33],
       [32, 50, 33, 42],
       [43, 52, 35, 53],
       [52, 35, 53, 36],
       [35, 53, 36, 45],
       [46, 55, 38, 56],
       [55, 38, 56, 39],
       [38, 56, 39, 48],
       [50, 59, 42, 60],


In [71]:
angles

array([[ 6, 15,  0, 16],
       [15,  0, 16,  1],
       [ 0, 16,  1,  8],
       [ 9, 18,  2, 19],
       [18,  2, 19,  3],
       [ 2, 19,  3, 11],
       [12, 21,  4, 22],
       [21,  4, 22,  5],
       [ 4, 22,  5, 14],
       [16, 25,  8, 26],
       [25,  8, 26,  9],
       [ 8, 26,  9, 18],
       [19, 28, 11, 29],
       [28, 11, 29, 12],
       [11, 29, 12, 21],
       [23, 32, 15, 33],
       [32, 15, 33, 16],
       [15, 33, 16, 25],
       [26, 35, 18, 36],
       [35, 18, 36, 19],
       [18, 36, 19, 28],
       [29, 38, 21, 39],
       [38, 21, 39, 22],
       [21, 39, 22, 31],
       [33, 42, 25, 43],
       [42, 25, 43, 26],
       [25, 43, 26, 35],
       [36, 45, 28, 46],
       [45, 28, 46, 29],
       [28, 46, 29, 38],
       [40, 49, 32, 50],
       [49, 32, 50, 33],
       [32, 50, 33, 42],
       [43, 52, 35, 53],
       [52, 35, 53, 36],
       [35, 53, 36, 45],
       [46, 55, 38, 56],
       [55, 38, 56, 39],
       [38, 56, 39, 48],
       [50, 59, 42, 60],


In [116]:
angles

array([[ 6, 15,  0, 16],
       [15,  0, 16,  1],
       [ 0, 16,  1,  8],
       [ 9, 18,  2, 19],
       [18,  2, 19,  3],
       [ 2, 19,  3, 11],
       [12, 21,  4, 22],
       [21,  4, 22,  5],
       [ 4, 22,  5, 14],
       [16, 25,  8, 26],
       [25,  8, 26,  9],
       [ 8, 26,  9, 18],
       [19, 28, 11, 29],
       [28, 11, 29, 12],
       [11, 29, 12, 21],
       [23, 32, 15, 33],
       [32, 15, 33, 16],
       [15, 33, 16, 25],
       [26, 34, 18, 35],
       [34, 18, 35, 19],
       [18, 35, 19, 28],
       [29, 36, 21, 37],
       [36, 21, 37, 22],
       [21, 37, 22, 31]])

In [117]:
other_angles

[[2, 9, 18, 26], [4, 12, 21, 29], [8, 16, 25, 33], [11, 19, 28, 35]]

In [118]:
np.vstack((angles, other_angles))

array([[ 6, 15,  0, 16],
       [15,  0, 16,  1],
       [ 0, 16,  1,  8],
       [ 9, 18,  2, 19],
       [18,  2, 19,  3],
       [ 2, 19,  3, 11],
       [12, 21,  4, 22],
       [21,  4, 22,  5],
       [ 4, 22,  5, 14],
       [16, 25,  8, 26],
       [25,  8, 26,  9],
       [ 8, 26,  9, 18],
       [19, 28, 11, 29],
       [28, 11, 29, 12],
       [11, 29, 12, 21],
       [23, 32, 15, 33],
       [32, 15, 33, 16],
       [15, 33, 16, 25],
       [26, 34, 18, 35],
       [34, 18, 35, 19],
       [18, 35, 19, 28],
       [29, 36, 21, 37],
       [36, 21, 37, 22],
       [21, 37, 22, 31],
       [ 2,  9, 18, 26],
       [ 4, 12, 21, 29],
       [ 8, 16, 25, 33],
       [11, 19, 28, 35]])

In [119]:
for i in np.vstack((angles, other_angles)):
    print(f"  {i[0]:<3}     {i[1]:<3}     {i[2]:<3}     {i[3]:<3}    180     400")
    

  6       15      0       16     180     400
  15      0       16      1      180     400
  0       16      1       8      180     400
  9       18      2       19     180     400
  18      2       19      3      180     400
  2       19      3       11     180     400
  12      21      4       22     180     400
  21      4       22      5      180     400
  4       22      5       14     180     400
  16      25      8       26     180     400
  25      8       26      9      180     400
  8       26      9       18     180     400
  19      28      11      29     180     400
  28      11      29      12     180     400
  11      29      12      21     180     400
  23      32      15      33     180     400
  32      15      33      16     180     400
  15      33      16      25     180     400
  26      34      18      35     180     400
  34      18      35      19     180     400
  18      35      19      28     180     400
  29      36      21      37     180     400
  36      